# TER Picopatt — Visualisation des 0

##  1) Importation des librairies et configuration des chemins

 Importation des librairies principales et définition des dossiers de travail.

In [9]:
from pathlib import Path
import re, unicodedata
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from zoneinfo import ZoneInfo  

# Dossiers de données et de sortie
DATA_DIR = Path("../picopatt-database/dataproc")
OUTPUT_DIR = Path("./outputs"); OUTPUT_DIR.mkdir(parents=True, exist_ok=True)
FIG_DIR = OUTPUT_DIR / "figures"; FIG_DIR.mkdir(parents=True, exist_ok=True)

# Affichage complet des colonnes
pd.set_option("display.max_columns", 200)

## 2) Fonctions utilitaires

Définition de fonctions pour normaliser les textes, extraire le nom du parcours, parser les dates et attribuer un créneau horaire ('M1'
'M4').

In [10]:
def strip_accents(s):
    """ 
    Supprime les accents d'une chaîne
    """ 
    if pd.isna(s): return s
    return ''.join(c for c in unicodedata.normalize('NFKD', str(s)) if not unicodedata.combining(c))

def normalize_track(x):
    """
    Nettoie et rassemble les noms de parcours
    """ 
    if pd.isna(x): return np.nan
    x = strip_accents(str(x)).lower().strip()
    x = x.replace("boulevard", "boulevards")
    if "antigone" in x: return "antigone"
    if "boulevards" in x: return "boulevards"
    if "ecusson" in x: return "ecusson"
    return np.nan

def infer_track_from_filename(name: str):
    """
    Déduit le parcours à partir du nom du fichier
    """  
    n = strip_accents(name.lower())
    if "antigone" in n:   return "antigone"
    if "boulevard" in n:  return "boulevards"
    if "ecusson" in n:    return "ecusson"
    return np.nan

def parse_fr_ts(s):
    """
    Convertit les dates au format français en 
    """
    return pd.to_datetime(s, errors="coerce", dayfirst=True)

def assign_mslot_dst(ts_series: pd.Series, tz_name: str = "Europe/Paris") -> pd.Series:
    """
    M1..M4 en tenant compte DST (hiver UTC+1, été UTC+2).
    Hiver :
      M1 08–11, M2 11–14, M3 14–17, M4 17–20
    Été :
      M1 09–12, M2 12–15, M3 15–18, M4 18–21
    Intervalles [début, fin)
    """
    s = pd.to_datetime(ts_series, errors="coerce", dayfirst=True)

    # Localisation Europe/Paris
    s_loc = s.dt.tz_localize(
        ZoneInfo(tz_name),
        ambiguous="infer",
        nonexistent="shift_forward"
    )

    # Récupère le décalage UTC pour détecter été/hiver
    off = s_loc.map(lambda x: x.utcoffset())
    is_summer = off >= pd.Timedelta(hours=2)

    hh = s_loc.dt.hour

    conds, labels = [], []
    # Hiver
    conds += [
        (~is_summer) & (hh.between(8,10)),   # 08:00..10:59
        (~is_summer) & (hh.between(11,13)),  # 11:00..13:59
        (~is_summer) & (hh.between(14,16)),  # 14:00..16:59
        (~is_summer) & (hh.between(17,19)),  # 17:00..19:59
    ]
    labels += ["M1","M2","M3","M4"]

    # Été
    conds += [
        (is_summer) & (hh.between(9,11)),    # 09:00..11:59
        (is_summer) & (hh.between(12,14)),   # 12:00..14:59
        (is_summer) & (hh.between(15,17)),   # 15:00..17:59
        (is_summer) & (hh.between(18,20)),   # 18:00..20:59
    ]
    labels += ["M1","M2","M3","M4"]

    result = np.select(conds, labels, default=None)

    return pd.Series(result, index=ts_series.index, dtype="object")


## 3) Chargement et préparation des données

Lecture de tous les fichiers de données ('.csv', '.xlsx') dans le dossier 'DATAproc', nettoyage et harmonisation des colonnes, puis ajout des colonnes 'M_slot' (créneau horaire) et 'date'.

Nous préparons et vérifions les données brutes avant toute analyse.

Comme elles viennent de plusieurs fichiers, plusieurs jours, parcours, on va  :

les fusionner proprement dans un seul tableau,
uniformiser les colonnes (mêmes noms, même format),
et vérifier la couverture temporelle et la répartition des mesures.

In [11]:
def read_any(p: Path) -> pd.DataFrame:
    """
    Lit un fichier (CSV ou Excel), en essayant plusieurs séparateurs.
    """
    if p.suffix.lower() in (".xlsx", ".xls"):
        df = pd.read_excel(p)
    else:
        df = None
        for sep in [",", ";", "\t"]:
            try:
                temp = pd.read_csv(p, sep=sep)
                if temp.shape[1] >= 5: 
                    df = temp
                    break
            except Exception:
                continue
        if df is None:
            raise ValueError(f"Impossible de lire le fichier : {p}")
    df["__source_file"] = p.name
    return df


def load_all(data_dir: Path) -> pd.DataFrame:
    """
    Charge et fusionne tous les fichiers du dossier en un seul DataFrame.
    """
    paths = sorted([p for p in data_dir.rglob("*") if p.suffix.lower() in (".csv", ".xlsx", ".xls")])
    assert paths, f"Aucun fichier trouvé dans {data_dir.resolve()}"

    frames = []
    for p in paths:
        df = read_any(p)

        # Conversion du timestamp
        if "timestamp" in df.columns:
            df["timestamp"] = parse_fr_ts(df["timestamp"])

        # Normalisation du nom de parcours à partir du nom du fichier
        trk_file = infer_track_from_filename(p.name)
        if "track_id" not in df.columns:
            df["track_id"] = trk_file
        else:
            df["track_id"] = df["track_id"].map(normalize_track)
            if df["track_id"].notna().any():
                df["track_id"] = df["track_id"].ffill().bfill()
            df["track_id"] = df["track_id"].fillna(trk_file)

        # Fusion des coordonnées GPS RTK et ontrack
        for c in ["lon_rtk", "lat_rtk", "lon_ontrack", "lat_ontrack"]:
            if c not in df.columns:
                df[c] = np.nan
        df["lon"] = df["lon_rtk"].where(df["lon_rtk"].notna(), df["lon_ontrack"])
        df["lat"] = df["lat_rtk"].where(df["lat_rtk"].notna(), df["lat_ontrack"])

        frames.append(df)

    return pd.concat(frames, ignore_index=True, sort=False)


# Chargement complet
raw = load_all(DATA_DIR)

ts_local = pd.to_datetime(raw["timestamp"], errors="coerce", dayfirst=True) \
               .dt.tz_localize(ZoneInfo("Europe/Paris"), ambiguous="infer", nonexistent="shift_forward")
raw["M_slot"] = assign_mslot_dst(raw["timestamp"])
raw["date"]   = ts_local.dt.date

# Résumé rapide
print("Couverture :", raw["timestamp"].min(), "->", raw["timestamp"].max())
print("Parcours :", raw["track_id"].dropna().unique())
print(
    raw.pivot_table(index="track_id", columns="M_slot",
                    values="timestamp", aggfunc="count")
       .fillna(0)
       .astype(int)
)

C:\Users\antoc\AppData\Local\Temp\ipykernel_38512\987342011.py:34: UserWarning: Parsing dates in %Y-%m-%d %H:%M:%S format when dayfirst=True was specified. Pass `dayfirst=False` or specify a format to silence this warning.
  return pd.to_datetime(s, errors="coerce", dayfirst=True)
C:\Users\antoc\AppData\Local\Temp\ipykernel_38512\987342011.py:34: UserWarning: Parsing dates in %Y-%m-%d %H:%M:%S format when dayfirst=True was specified. Pass `dayfirst=False` or specify a format to silence this warning.
  return pd.to_datetime(s, errors="coerce", dayfirst=True)
C:\Users\antoc\AppData\Local\Temp\ipykernel_38512\987342011.py:34: UserWarning: Parsing dates in %Y-%m-%d %H:%M:%S format when dayfirst=True was specified. Pass `dayfirst=False` or specify a format to silence this warning.
  return pd.to_datetime(s, errors="coerce", dayfirst=True)
C:\Users\antoc\AppData\Local\Temp\ipykernel_38512\987342011.py:34: UserWarning: Parsing dates in %Y-%m-%d %H:%M:%S format when dayfirst=True was specified

Couverture : 2024-03-12 08:30:07 -> 2025-09-01 19:08:20
Parcours : ['antigone' 'boulevards' 'ecusson']
M_slot         M1     M2     M3     M4
track_id                              
antigone    29675  30263  31419  26696
boulevards  27959  33312  26151  27108
ecusson     30435  27041  26656  19730


Identification sur l'ensemble de données des valeurs zéro dans les colonnes numériques 

In [13]:
numerical_cols = raw.select_dtypes(include=np.number).columns.tolist()

zero_percentages_global = {}

# Boucle sur toutes les colonnes numériques
for col in numerical_cols:
    total_non_missing = raw[col].count()
    if total_non_missing > 0:
        zero_count = (raw[col] == 0).sum()
        zero_percentage = (zero_count / total_non_missing) * 100
        zero_percentages_global[col] = zero_percentage
    else:
        zero_percentages_global[col] = np.nan

# Trie les colonnes par pourcentage décroissant
sorted_zero_global = sorted(zero_percentages_global.items(), key=lambda x: x[1], reverse=True)

# Affiche les résultats
print("Pourcentage de valeurs nulles pour chaque variable numérique")
for col, pct in sorted_zero_global:
    print(f"{col}: {pct:.2f}%")

Pourcentage de valeurs nulles pour chaque variable numérique
sw_down: 76.15%
sw_left: 49.38%
sw_up: 26.01%
ws: 4.11%
sw_back: 1.17%
tair_tc2: 0.59%
wdir: 0.55%
sw_right: 0.00%
section_id: 0.00%
segment_id: 0.00%
point_id: 0.00%
section_duration: 0.00%
section_speed: 0.00%
gnss_accuracy: 0.00%
lon_ontrack: 0.00%
lat_ontrack: 0.00%
lon_rtk: 0.00%
lat_rtk: 0.00%
sun_azimuth: 0.00%
sun_height: 0.00%
tair_thermohygro: 0.00%
tair_tc1: 0.00%
tair_anemo: 0.00%
rh_thermohygro: 0.00%
sw_front: 0.00%
lw_up: 0.00%
lw_down: 0.00%
lw_front: 0.00%
lw_back: 0.00%
lw_left: 0.00%
lw_right: 0.00%
tmrt: 0.00%
pet: 0.00%
lon: 0.00%
lat: 0.00%


In [14]:
# Pour chaque parcours 

for track_id, df_track in raw.groupby("track_id"):
    zero_percentages = {}
    for col in numerical_cols:
        total_non_missing = df_track[col].count()
        if total_non_missing > 0:
            zero_count = (df_track[col] == 0).sum()
            zero_percentage = (zero_count / total_non_missing) * 100
            zero_percentages[col] = zero_percentage
        else:
            zero_percentages[col] = 0

    sorted_zero_percentages = sorted(zero_percentages.items(), key=lambda item: item[1], reverse=True)

    print(f"\nPourcentage de valeurs nulles pour les colonnes numériques dans le parcours {track_id}")
    for col, percentage in sorted_zero_percentages:
        print(f"{col}: {percentage:.2f}%")



Pourcentage de valeurs nulles pour les colonnes numériques dans le parcours antigone
sw_down: 64.87%
sw_left: 33.70%
sw_up: 26.23%
ws: 2.78%
sw_back: 2.74%
wdir: 0.31%
sw_right: 0.00%
section_id: 0.00%
segment_id: 0.00%
point_id: 0.00%
section_duration: 0.00%
section_speed: 0.00%
gnss_accuracy: 0.00%
lon_ontrack: 0.00%
lat_ontrack: 0.00%
lon_rtk: 0.00%
lat_rtk: 0.00%
sun_azimuth: 0.00%
sun_height: 0.00%
tair_thermohygro: 0.00%
tair_tc1: 0.00%
tair_tc2: 0.00%
tair_anemo: 0.00%
rh_thermohygro: 0.00%
sw_front: 0.00%
lw_up: 0.00%
lw_down: 0.00%
lw_front: 0.00%
lw_back: 0.00%
lw_left: 0.00%
lw_right: 0.00%
tmrt: 0.00%
pet: 0.00%
lon: 0.00%
lat: 0.00%

Pourcentage de valeurs nulles pour les colonnes numériques dans le parcours boulevards
sw_down: 75.67%
sw_left: 50.70%
sw_up: 27.11%
ws: 5.02%
wdir: 0.66%
section_id: 0.00%
segment_id: 0.00%
point_id: 0.00%
section_duration: 0.00%
section_speed: 0.00%
gnss_accuracy: 0.00%
lon_ontrack: 0.00%
lat_ontrack: 0.00%
lon_rtk: 0.00%
lat_rtk: 0.00%
sun

Deux variables ont un pourcentage trop élevé sur l'ensemble des données

In [ ]:
# Drop colonnes avec beaucoup de 0 
raw = raw.drop(columns=['sw_down','sw_left'], errors='ignore') 

In [6]:
# Zéros par parcours

def zeros_report_for_track(raw: pd.DataFrame, track_name: str, cols=None) -> pd.DataFrame:
    sub = raw.loc[raw["track_id"].astype(str).str.strip().str.lower() == track_name.lower()].copy()
    if sub.empty:
        print(f"Aucun enregistrement pour {track_name}")
        return pd.DataFrame()

    if "__row_in_file" not in sub.columns:
        sub["__row_in_file"] = sub.groupby("__source_file").cumcount()

    if "date" not in sub.columns and "timestamp" in sub.columns and pd.api.types.is_datetime64_any_dtype(sub["timestamp"]):
        sub["date"] = sub["timestamp"].dt.date

    if cols is None:
        exclude = {
            "point_id","segment_id","section_id","track_id","project_id",
            "timestamp","date","M_slot","passage_slot","__source_file",
            "lon","lat","lon_rtk","lat_rtk","lon_ontrack","lat_ontrack",
            "__row_in_file"
        }
        num_cols = sub.select_dtypes(include=[np.number]).columns
        cols = [c for c in num_cols if c not in exclude]
    else:
        cols = [c for c in cols if c in sub.columns]
    if not cols:
        return pd.DataFrame()

    is_zero = (sub[cols] == 0)
    long = is_zero.stack().rename("is_zero").reset_index()
    long = long[long["is_zero"]]
    long = long.rename(columns={"level_1": "variable"}).drop(columns=["is_zero"])

    meta_cols = [c for c in ["timestamp","date","track_id","__source_file","__row_in_file"] if c in sub.columns]
    out = long.merge(sub[meta_cols], left_on="level_0", right_index=True, how="left").drop(columns=["level_0"])

    wanted = ["track_id","variable","timestamp","date","__source_file","__row_in_file"]
    wanted = [c for c in wanted if c in out.columns]
    out = out[wanted].sort_values(["__source_file","__row_in_file","variable"]).reset_index(drop=True)
    return out

# Génération & export
tracks_focus = ["antigone", "ecusson", "boulevards"]
reports = {}
for t in tracks_focus:
    rep = zeros_report_for_track(raw, t)
    reports[t] = rep
    out_csv = OUTPUT_DIR / f"zeros_{t}.csv"
    rep.to_csv(out_csv, index=False)
    print(f"Export zéros {t}: {out_csv} ({len(rep)} lignes)")
    if not rep.empty:
        counts = (
            rep.groupby(["__source_file","variable"])
              .size().reset_index(name="n_zero")
              .sort_values(["__source_file","n_zero"], ascending=[True, False])
        )
        display(f"Résumé zéros - {t}", counts.head(30))


Export zéros antigone: outputs\zeros_antigone.csv (38940 lignes)


'Résumé zéros - antigone'

,__source_file,variable,n_zero
3,picopatt_montpellier_antigone_20241029_0835.csv,ws,122
0,picopatt_montpellier_antigone_20241029_0835.csv,sw_back,19
2,picopatt_montpellier_antigone_20241029_0835.csv,wdir,4
1,picopatt_montpellier_antigone_20241029_0835.csv,sw_up,3
5,picopatt_montpellier_antigone_20241029_1127.csv,ws,100
4,picopatt_montpellier_antigone_20241029_1127.csv,wdir,12
7,picopatt_montpellier_antigone_20241029_1423.csv,ws,107
6,picopatt_montpellier_antigone_20241029_1423.csv,wdir,1
9,picopatt_montpellier_antigone_20241029_1722.csv,sw_up,4390
8,picopatt_montpellier_antigone_20241029_1722.csv,sw_back,1543


Export zéros ecusson: outputs\zeros_ecusson.csv (33675 lignes)


'Résumé zéros - ecusson'

,__source_file,variable,n_zero
3,picopatt_montpellier_ecusson_20241031_0827.csv,ws,267
1,picopatt_montpellier_ecusson_20241031_0827.csv,sw_up,57
2,picopatt_montpellier_ecusson_20241031_0827.csv,wdir,26
0,picopatt_montpellier_ecusson_20241031_0827.csv,sw_back,9
5,picopatt_montpellier_ecusson_20241031_1127.csv,ws,251
4,picopatt_montpellier_ecusson_20241031_1127.csv,wdir,27
6,picopatt_montpellier_ecusson_20241031_1425.csv,sw_up,611
8,picopatt_montpellier_ecusson_20241031_1425.csv,ws,197
7,picopatt_montpellier_ecusson_20241031_1425.csv,wdir,9
9,picopatt_montpellier_ecusson_20241112_0830.csv,sw_up,416


Export zéros boulevards: outputs\zeros_boulevards.csv (38032 lignes)


'Résumé zéros - boulevards'

,__source_file,variable,n_zero
2,picopatt_montpellier_boulevards_20241114_0828.csv,ws,226
0,picopatt_montpellier_boulevards_20241114_0828.csv,sw_up,51
1,picopatt_montpellier_boulevards_20241114_0828.csv,wdir,49
4,picopatt_montpellier_boulevards_20241114_1121.csv,ws,116
3,picopatt_montpellier_boulevards_20241114_1121.csv,wdir,91
7,picopatt_montpellier_boulevards_20241114_1429.csv,ws,164
5,picopatt_montpellier_boulevards_20241114_1429.csv,sw_up,78
6,picopatt_montpellier_boulevards_20241114_1429.csv,wdir,76
8,picopatt_montpellier_boulevards_20241114_1728.csv,sw_up,4119
10,picopatt_montpellier_boulevards_20241114_1728.csv,ws,135


In [7]:
# Intervalles de zéros consécutifs

def zeros_intervals_for_track(raw: pd.DataFrame, track_name: str, cols=None, min_len: int = 1) -> pd.DataFrame:
    sub = raw.loc[raw["track_id"].astype(str).str.strip().str.lower() == track_name.lower()].copy()
    if sub.empty:
        return pd.DataFrame()

    if "__row_in_file" not in sub.columns:
        sub["__row_in_file"] = sub.groupby("__source_file").cumcount()

    if cols is None:
        exclude = {
            "point_id","segment_id","section_id","track_id","project_id",
            "timestamp","date","M_slot","passage_slot","__source_file",
            "lon","lat","lon_rtk","lat_rtk","lon_ontrack","lat_ontrack",
            "__row_in_file"
        }
        num_cols = sub.select_dtypes(include=[np.number]).columns
        cols = [c for c in num_cols if c not in exclude]
    else:
        cols = [c for c in cols if c in sub.columns]
    if not cols:
        return pd.DataFrame()

    out_rows = []
    has_ts = "timestamp" in sub.columns
    for src, chunk in sub.groupby("__source_file", sort=False):
        # ordre stable
        sort_keys = [c for c in ("timestamp","__row_in_file") if c in chunk.columns]
        if sort_keys:
            chunk = chunk.sort_values(sort_keys, kind="stable")
        for var in cols:
            mask = (chunk[var] == 0)
            if not mask.any():
                continue
            run_id = (mask != mask.shift(fill_value=False)).cumsum()
            for _, g in chunk.loc[mask].groupby(run_id[mask]):
                n = len(g)
                if n < min_len:
                    continue
                out_rows.append({
                    "track_id": track_name,
                    "variable": var,
                    "__source_file": src,
                    "start_row_in_file": int(g["__row_in_file"].iloc[0]),
                    "end_row_in_file":   int(g["__row_in_file"].iloc[-1]),
                    "n_rows": int(n),
                    "start_timestamp": g["timestamp"].iloc[0] if has_ts else None,
                    "end_timestamp":   g["timestamp"].iloc[-1] if has_ts else None,
                })
    out = pd.DataFrame(out_rows)
    if not out.empty:
        out = out.sort_values(["__source_file","variable","start_row_in_file"]).reset_index(drop=True)
    return out

# Génération & export
intervals = {}
for t in tracks_focus:
    df_int = zeros_intervals_for_track(raw, t, min_len=1)
    intervals[t] = df_int
    out_csv = OUTPUT_DIR / f"zeros_intervalles_{t}.csv"
    df_int.to_csv(out_csv, index=False)
    print(f"Intervalles {t}: {out_csv} ({len(df_int)} lignes)")
    if not df_int.empty:
        counts_int = (
            df_int.groupby(["__source_file","variable"])["n_rows"]
                 .agg(nb_intervalles="count", total_lignes_zero="sum")
                 .reset_index()
                 .sort_values(["__source_file","total_lignes_zero"], ascending=[True, False])
        )
        display(f"Résumé intervalles — {t}", counts_int.head(30))


Intervalles antigone: outputs\zeros_intervalles_antigone.csv (2617 lignes)


'Résumé intervalles — antigone'

,__source_file,variable,nb_intervalles,total_lignes_zero
3,picopatt_montpellier_antigone_20241029_0835.csv,ws,28,122
0,picopatt_montpellier_antigone_20241029_0835.csv,sw_back,18,19
2,picopatt_montpellier_antigone_20241029_0835.csv,wdir,3,4
1,picopatt_montpellier_antigone_20241029_0835.csv,sw_up,3,3
5,picopatt_montpellier_antigone_20241029_1127.csv,ws,37,100
4,picopatt_montpellier_antigone_20241029_1127.csv,wdir,11,12
7,picopatt_montpellier_antigone_20241029_1423.csv,ws,37,107
6,picopatt_montpellier_antigone_20241029_1423.csv,wdir,1,1
9,picopatt_montpellier_antigone_20241029_1722.csv,sw_up,1,4390
8,picopatt_montpellier_antigone_20241029_1722.csv,sw_back,456,1543


Intervalles ecusson: outputs\zeros_intervalles_ecusson.csv (2520 lignes)


'Résumé intervalles — ecusson'

,__source_file,variable,nb_intervalles,total_lignes_zero
3,picopatt_montpellier_ecusson_20241031_0827.csv,ws,73,267
1,picopatt_montpellier_ecusson_20241031_0827.csv,sw_up,11,57
2,picopatt_montpellier_ecusson_20241031_0827.csv,wdir,20,26
0,picopatt_montpellier_ecusson_20241031_0827.csv,sw_back,9,9
5,picopatt_montpellier_ecusson_20241031_1127.csv,ws,79,251
4,picopatt_montpellier_ecusson_20241031_1127.csv,wdir,22,27
6,picopatt_montpellier_ecusson_20241031_1425.csv,sw_up,20,611
8,picopatt_montpellier_ecusson_20241031_1425.csv,ws,60,197
7,picopatt_montpellier_ecusson_20241031_1425.csv,wdir,8,9
9,picopatt_montpellier_ecusson_20241112_0830.csv,sw_up,21,416


Intervalles boulevards: outputs\zeros_intervalles_boulevards.csv (2316 lignes)


'Résumé intervalles — boulevards'

,__source_file,variable,nb_intervalles,total_lignes_zero
2,picopatt_montpellier_boulevards_20241114_0828.csv,ws,89,226
0,picopatt_montpellier_boulevards_20241114_0828.csv,sw_up,5,51
1,picopatt_montpellier_boulevards_20241114_0828.csv,wdir,44,49
4,picopatt_montpellier_boulevards_20241114_1121.csv,ws,57,116
3,picopatt_montpellier_boulevards_20241114_1121.csv,wdir,72,91
7,picopatt_montpellier_boulevards_20241114_1429.csv,ws,75,164
5,picopatt_montpellier_boulevards_20241114_1429.csv,sw_up,6,78
6,picopatt_montpellier_boulevards_20241114_1429.csv,wdir,56,76
8,picopatt_montpellier_boulevards_20241114_1728.csv,sw_up,1,4119
10,picopatt_montpellier_boulevards_20241114_1728.csv,ws,32,135


In [8]:
# Charge tous les rapports "zeros_*.csv" présents (par parcours)
reports = {}
for p in OUTPUT_DIR.glob("zeros_*.csv"):
    if p.name.startswith("zeros_intervalles_"):  # on veut pas les intervalles ici
        continue
    df = pd.read_csv(p)
    track = p.stem.replace("zeros_", "")
    reports[track] = df


In [10]:
# Variables à traiter uniquement
VARS_TO_IMPUTE = ("sw_up","sw_back","tair_tc2")
MAX_RUN = 30

def _replace_zero_runs_with_endpoints_mean(s: pd.Series, max_run: int = 30) -> pd.Series:
    """
    Remplace les runs de 0 par la moyenne (valeur[i0-1] + valeur[i1+1]) / 2
    si et seulement si:
      - longueur du run <= max_run
      - les deux extrémités existent et ne sont pas NaN
    Sinon, met NaN sur tout le run.

    s doit être numérique; les NaN sont propagés.
    """
    x = pd.to_numeric(s, errors="coerce").copy()
    v = x.values
    n = len(v)
    if n == 0:
        return x

    mask = (v == 0)
    if not mask.any():
        return x

    # détecter starts/ends de runs True dans mask
    # start: position True avec précédent False (ou début)
    # end  : position True avec suivant False (ou fin)
    prev = np.pad(mask[:-1], (1,0), constant_values=False)
    next_ = np.pad(mask[1:],  (0,1), constant_values=False)
    starts = np.where(mask & ~prev)[0]
    ends   = np.where(mask & ~next_)[0]

    for i0, i1 in zip(starts, ends):
        run_len = i1 - i0 + 1
        if run_len <= max_run and i0 > 0 and i1 < n - 1:
            left  = v[i0 - 1]
            right = v[i1 + 1]
            if np.isfinite(left) and np.isfinite(right):
                fill_val = (left + right) / 2.0
                v[i0:i1+1] = fill_val
            else:
                v[i0:i1+1] = np.nan
        else:
            # trop long, ou bord du fichier, ou extrémité manquante/non numérique
            v[i0:i1+1] = np.nan

    return pd.Series(v, index=x.index)

In [14]:
# Dossiers de sortie
CLEAN_DIR = OUTPUT_DIR / "clean_nozeros"
CLEAN_DIR.mkdir(parents=True, exist_ok=True)

def process_file_block(df_file: pd.DataFrame,
                       vars_to_impute=VARS_TO_IMPUTE,
                       max_run: int = MAX_RUN) -> tuple[pd.DataFrame, list[dict]]:
    """
    Applique l'imputation "moyenne des voisins" sur les colonnes cibles d'un fichier.
    Retourne le DataFrame transformé + un résumé des traitements.
    """
    df_out = df_file.copy()
    summary = []

    sort_keys = [c for c in ("timestamp","__row_in_file") if c in df_out.columns]
    if sort_keys:
        df_out = df_out.sort_values(sort_keys, kind="stable")

    for col in vars_to_impute:
        if col not in df_out.columns:
            summary.append({"__source_file": df_out["__source_file"].iloc[0],
                            "variable": col, "n_runs_replaced": 0, "n_runs_nan": 0,
                            "n_points_replaced": 0, "n_points_nan": 0, "note": "colonne absente"})
            continue

        # Numériser la colonne
        df_out[col] = pd.to_numeric(df_out[col], errors="coerce")

        # Compter avant
        was_zero = (df_out[col] == 0)
        n_zero_before = int(was_zero.sum())

        # Appliquer imputation run par run sur une copie
        col_before = df_out[col].copy()
        col_after  = _replace_zero_runs_with_endpoints_mean(col_before, max_run=max_run)
        df_out[col] = col_after

        # Bilan des runs
        mask0 = (col_before.values == 0)
        prev = np.pad(mask0[:-1], (1,0), constant_values=False)
        next_ = np.pad(mask0[1:],  (0,1), constant_values=False)
        starts = np.where(mask0 & ~prev)[0]
        ends   = np.where(mask0 & ~next_)[0]

        n_runs_replaced = 0
        n_runs_nan = 0
        n_points_replaced = 0
        n_points_nan = 0

        for i0, i1 in zip(starts, ends):
            run_len = i1 - i0 + 1
            block_after = col_after.iloc[i0:i1+1]
            # Si tout le bloc n'est plus NaN => remplacé par une constante (la moyenne)
            if block_after.notna().all():
                n_runs_replaced += 1
                n_points_replaced += run_len
            else:
                # tout le run doit être NaN
                n_runs_nan += 1
                n_points_nan += run_len

        summary.append({
            "__source_file": df_out["__source_file"].iloc[0],
            "variable": col,
            "n_runs_replaced": int(n_runs_replaced),
            "n_runs_nan": int(n_runs_nan),
            "n_points_replaced": int(n_points_replaced),
            "n_points_nan": int(n_points_nan),
            "n_zero_before": int(n_zero_before)
        })

    return df_out, summary

# Boucle globale par fichier
all_summ = []    
for src, df_file in raw.groupby("__source_file", sort=False):
    df_clean, summ = process_file_block(df_file, vars_to_impute=VARS_TO_IMPUTE, max_run=MAX_RUN)
    all_summ.extend(summ)
    out_path = CLEAN_DIR / src
    df_clean.to_csv(out_path, index=False)

# Export du résumé global
summary_df = pd.DataFrame(all_summ)
summary_path = OUTPUT_DIR / "visu_zeros/imputation_summary.csv"
summary_df.to_csv(summary_path, index=False)

display(summary_df.head(30))
print(f"Fichiers nettoyés écrits dans: {CLEAN_DIR}")
print(f"Résumé: {summary_path}")

,__source_file,variable,n_runs_replaced,n_runs_nan,n_points_replaced,n_points_nan,n_zero_before
0,picopatt_montpellier_antigone_20241029_0835.csv,sw_up,3,0,3,0,3
1,picopatt_montpellier_antigone_20241029_0835.csv,sw_back,18,0,19,0,19
2,picopatt_montpellier_antigone_20241029_0835.csv,tair_tc2,0,0,0,0,0
3,picopatt_montpellier_antigone_20241029_1127.csv,sw_up,0,0,0,0,0
4,picopatt_montpellier_antigone_20241029_1127.csv,sw_back,0,0,0,0,0
5,picopatt_montpellier_antigone_20241029_1127.csv,tair_tc2,0,0,0,0,0
6,picopatt_montpellier_antigone_20241029_1423.csv,sw_up,0,0,0,0,0
7,picopatt_montpellier_antigone_20241029_1423.csv,sw_back,0,0,0,0,0
8,picopatt_montpellier_antigone_20241029_1423.csv,tair_tc2,0,0,0,0,0
9,picopatt_montpellier_antigone_20241029_1722.csv,sw_up,0,1,0,4390,4390


Fichiers nettoyés écrits dans: outputs\clean_nozeros
Résumé: outputs\visu_zeros\imputation_summary.csv
